# MLP with COMPAS Dataset - mk1
#### 17 May 2022

## Load Dependencies <a id="libraries"></a>

In [2]:
import pandas as pd
import numpy as np
import timeit
import os
import math

In [3]:
import torch
import torch.nn as nn
import torchmetrics
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
import warnings
warnings.filterwarnings('ignore')

### Notebook options and parameters

In [6]:
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250

In [7]:
# random seed for consistency in pytorch and numpy
SEED = 23

torch.manual_seed(SEED)
np.random.seed(SEED)

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Hyperparameter options

In [9]:
# activation functions
elu = nn.ELU()
leaky = nn.LeakyReLU()
relu = nn.ReLU()
tanh = nn.Tanh()
sigmoid = nn.Sigmoid()

# loss functions
mse = nn.MSELoss()
bce = nn.BCELoss()
kld = nn.KLDivLoss()

# optimizers
adam = torch.optim.Adam
sgd = torch.optim.SGD

## Custom Functions <a id="functions"></a>

#### Create dataset for DataLoader

In [10]:
class dataset(Dataset):
    def __init__(self, data, target):
        self.X = torch.tensor(data, dtype=torch.float32, device=device)
        self.y = torch.tensor(target, dtype=torch.float32, device=device)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __len__(self):
        return len(self.X)

#### Training function

In [11]:
def train(data, model, criterion, metric, optimizer):
    size = len(data.dataset)
    batches = len(data)
    running_loss = []
    acc = 0.0
    correct = 0
    predictions = []
    actuals = []
    loss = 0.0
    acc = 0.0
    current = 0
    
    for batch, (X, y) in enumerate(data,1):
        
        # forward step
        optimizer.zero_grad() # zero out the gradients
        y_hat = model(X)  #forward pass
        
        y = y.unsqueeze(-1) # reduce dimensions of tensor from [256, 1] to [256]
        
        loss = criterion(y_hat, y)  # calculate loss
       
        #  backprop step
        loss.backward()       # backward pass through model; computes gradients
        optimizer.step()      # update weights
        
        # metrics
        y = y.int()
        acc = metric(y_hat, y)

        y_hat = y_hat.cpu().detach().numpy()
        y_hat = y_hat.reshape(-1)
        
        y = y.cpu().detach().numpy()
        y = y.reshape(-1)
        
        predictions.append(y_hat)
        actuals.append(y)
            
        running_loss.append(loss.item())
    
        ## every 50 batches, store the results
        #if batch % 50 == 0:
        #    current = batch * len(X)
        
    print(f"\tTraining\tAccuracy: {acc:1.10f}\tLoss: {loss:1.10f}")
    
    return predictions, running_loss

#### Testing function

In [12]:
def test(data, model, criterion, metric):
    size = len(data.dataset)
    batches = len(data)
    tst_loss = 0
    correct = 0
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for X, y in data:
            y_hat = model(X)
            y = y.unsqueeze(-1)
            tst_loss += criterion(y_hat, y).item()
            
            y = y.int()
            acc = metric(y_hat, y)
            #acc = acc.cpu().detach().numpy()
            
            y_hat = y_hat.cpu().detach().numpy()
            y_hat = y_hat.reshape(-1)
        
            y = y.cpu().detach().numpy()
            y = y.reshape(-1)
        
            predictions.append(y_hat)
            actuals.append(y)
            
    tst_loss /= batches
    correct /= size
    
    print(f"\tTest error\tAccuracy: {acc:1.10f}\tLoss: {tst_loss:1.10f}")
    return predictions, tst_loss, acc

#### Evaluation - train and test

In [13]:
def evaluate(training_data, test_data, model, loss_function, metric, optimizer, epochs, early_stop=True):
    training_results = []
    training_actuals = []
    training_losses = []

    testing_results = []
    testing_actuals = []

    start = timeit.default_timer()
    
    # early stop criteria
    
    default_patience = 3
    patience = default_patience
    running_loss = []
    running_acc = []

    for epoch in range(1,epochs+1):
        print (f"Epoch {epoch} / {epochs}")

        trng_result, trng_loss = train(training_data, model, loss_function, metric, optimizer)
        tst_result, tst_loss, acc = test(test_data, model, loss_function, metric)
        
        training_results.append(trng_result)
        training_losses.append(trng_loss)
        testing_results.append(tst_result)
        running_loss.append(tst_loss)
        running_acc.append(acc)
        
        # check for early stop
        if epoch > 3:            
            if early_stop and ((tst_loss >= running_loss[-2]) or math.isclose(acc,running_acc[-2], abs_tol=1e-8)):
                patience = patience - 1
            elif early_stop and (tst_loss < running_loss[-2]):
                patience = default_patience

            if early_stop and (patience <= 0):
                print("Early stop - ", end="")
                break

    
    print("Finished")
    print("* " * 30)
    stop = timeit.default_timer()
    print(f"Execution time (in seconds): {stop - start}")

    return training_results, training_losses, testing_results

#### Multilayer Perception (MLP)

In [14]:
class MLP(nn.Module):
    
    def __init__(self, in_size, hidden1=relu, hidden2=relu, hidden3=relu, out_layer=sigmoid):
        super(MLP, self).__init__()
        
        # construct model dependent on size of inputs
        out1_2 = 2*in_size // 3
        out2_3 = 2*out1_2 // 3
        out3_4 = 2*out2_3 // 3

        self.linear_stack = nn.Sequential(
            nn.Linear(in_size, out1_2),
            hidden1,
            nn.Linear(out1_2, out2_3),
            hidden2,
            nn.Linear(out2_3, out3_4),
            hidden3,
            nn.Linear(out3_4, 1),
            out_layer
        )
        
    def forward(self, x):
        out = self.linear_stack(x)

        return out

#### Flatten function

In [15]:
def flatten(thing):
    '''
    thing: a list
    
    flatten receives a multi-level list and flatten it down 2 layers
    '''
    #thing = [element for sublist in thing for element in sublist]
    return [element for sublist in thing for element in sublist]

#### Prepare data for eval and analysis

In [16]:
def prep(name, training_results, testing_results):
    trng_preds = [[] for _ in range(len(training_results))]

    tst_preds = [[] for _ in range(len(testing_results))]
    
    for i in range(len(training_results)):
        trng_preds[i] = flatten(training_results[i])

        tst_preds[i] = flatten(testing_results[i])

    return trng_preds, tst_preds

#### Send results to tensorboard

In [17]:
def to_tensorboard(tailend, directory, dataset, train_preds, test_preds, y_train, y_test):
    acc_trng_relu = []
    acc_tst_relu = []
    loss_relu = []

    writer = SummaryWriter(log_dir=directory)

    for i in range(len(train_preds)):
        y_hat = np.array(np.round(train_preds[i]))
        y_hat_tst = np.array(np.round(test_preds[i]))
        trng_acc = (y_hat == y_train).sum()/len(train_preds[i])
        test_acc = (y_hat_tst == y_test).sum()/len(test_preds[i])

        avg_loss = np.average(training_losses[i])

        acc_trng_relu.append(trng_acc)
        acc_tst_relu.append(test_acc)
        loss_relu.append(avg_loss)
        writer.add_scalar(f"Training Accuracy {tailend}", trng_acc, i)
        writer.add_scalar(f"Test Accuracy {tailend}", test_acc, i)
        writer.add_scalar(f"Training Loss {tailend}", avg_loss, i)
        writer.add_pr_curve(f"Precision-Recall Curve {tailend}", y_test, y_hat_tst, i)

    t, _ = next(iter(dataset))
    writer.add_graph(model, t)

    writer.close()

#### Save model

In [18]:
def custom_save(name, data, kind=1):
    '''
    name : string
        designated filename
    data : data or pytorch model
        the data to save
    kind : int
        sentinel value - 1 if pytorch model, 0 otherwise
    
    custom_save stores the data passed into the function into a file with the provided name
    '''
    
    if kind == 1:
        ex = ".pth"
    else:
        ex = ".parquet"
    
    sentinel = True
    i = 1

    while sentinel:
        dirlist = os.listdir()

        if name not in dirlist:
            if kind == 1:
                torch.save(data, name)
            else:
                data.to_parquet(name)
            print(f"{name} has been saved.")                
            sentinel = False
        if name in dirlist:
            print(f"{name} already exists.", end=" ")
            temp, ext = name.split(ex)
            if "_v" in temp:
                temp, _ = temp.split("_v")
            name = f"{temp}_v{i}{ex}"
            i = i + 1
            print(f"Changing file name to: {name}")

# Import Data<a id="data"></a>
##### COMPAS dataset

#### Uncomment next when using google colab

In [19]:
#data = pd.read_parquet("https://github.com/ollin23/bias/blob/main/compas_mk1_v1.parquet?raw=true")

#### Use the following cell if the data is stored locally

In [20]:
data = pd.read_parquet("compas_mk1_v1.parquet")

#### Peak the data

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6907 entries, 0 to 7213
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6907 non-null   int64  
 1   juv_fel_count            6907 non-null   int64  
 2   decile_score             6907 non-null   int64  
 3   juv_misd_count           6907 non-null   int64  
 4   juv_other_count          6907 non-null   int64  
 5   priors_count             6907 non-null   int64  
 6   days_b_screening_arrest  6907 non-null   float64
 7   c_days_from_compas       6907 non-null   float64
 8   is_recid                 6907 non-null   int64  
 9   is_violent_recid         6907 non-null   int64  
 10  decile_score.1           6907 non-null   int64  
 11  v_decile_score           6907 non-null   int64  
 12  priors_count.1           6907 non-null   int64  
 13  start                    6907 non-null   int64  
 14  end                     

In [22]:
data.shape

(6907, 34)

In [23]:
data.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target
0,69,0,1,0,0,0,-1.0,1.0,0,0,1,1,0,0,327,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0
1,34,0,3,0,0,0,-1.0,1.0,1,1,3,1,0,9,159,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,1,1
2,24,0,4,0,1,4,-1.0,1.0,1,0,4,3,4,0,63,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,1,0,0,1
5,44,0,1,0,0,0,0.0,0.0,0,0,1,1,0,1,853,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0
6,41,0,6,0,0,14,-1.0,1.0,1,0,6,2,14,5,40,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,1,1


In [24]:
list(data.columns)

['age',
 'juv_fel_count',
 'decile_score',
 'juv_misd_count',
 'juv_other_count',
 'priors_count',
 'days_b_screening_arrest',
 'c_days_from_compas',
 'is_recid',
 'is_violent_recid',
 'decile_score.1',
 'v_decile_score',
 'priors_count.1',
 'start',
 'end',
 'sex',
 'age_cat_25to45',
 'age_cat_over45',
 'age_cat_under25',
 'race_black',
 'race_asian',
 'race_white',
 'race_hispanic',
 'race_native',
 'race_Other',
 'c_charge_degree',
 'score_text_High',
 'score_text_Low',
 'score_text_Medium',
 'v_score_text_High',
 'v_score_text_Low',
 'v_score_text_Medium',
 'event',
 'target']

### Process Data for Neural Net Usage <a id="convert"></a>

#### Split features into X and y

In [25]:
X = data.drop("target", axis=1).values.astype(np.float32)
y = data.target.values.astype(np.float32)

#### Standardize data

In [26]:
scaler = StandardScaler()
scaledX = scaler.fit_transform(X)

#### Split into training and testing sets

In [27]:
X_train, X_test, y_train, y_test = train_test_split(scaledX, y, test_size=0.3,
                                                        random_state=SEED, stratify=y)

In [28]:
X_train.shape

(4834, 33)

In [29]:
X_test.shape

(2073, 33)

#### Make datasets

In [30]:
scaled_train = dataset(X_train, y_train)
scaled_test = dataset(X_test, y_test)

##### Construct test results dataframe for persistent storage

In [31]:
features = list(data.drop("target", axis=1).columns)

temp = np.round(scaler.inverse_transform(X_test)).astype(int)
df = pd.DataFrame(temp, columns=features)
df["target"] = y_test

In [32]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1.0
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1.0
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0.0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1.0
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1.0


##### Note
For this dataset: Female == 1, Male == 0

# Experiment Phase 3 - COMPAS Data <a id="experiments"></a>

## Trial 3.1 - ReLU (COMPAS) <a id="relu"></a>

### Preliminaries

In [34]:
suffix = "mk2"
directory = f"COMPAS_{suffix}"
prefix = "compas"

#### Hyperparameters <a id="hyperparameters"></a>

In [35]:
alpha = 1e-5 # 0.00001
epochs = 150
batches = 100
hidden1 = relu
hidden2 = relu
hidden3 = relu
out_layer = sigmoid

#### DataLoader

In [36]:
# Helper function to feed data into pytorch neural network, only needs to execute once
training_s = DataLoader(scaled_train, batch_size=batches)
test_s = DataLoader(scaled_test, batch_size=batches)

#### Construct Model

In [37]:
# instantiate model
model = MLP(X.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=33, out_features=22, bias=True)
    (1): ReLU()
    (2): Linear(in_features=22, out_features=14, bias=True)
    (3): ReLU()
    (4): Linear(in_features=14, out_features=9, bias=True)
    (5): ReLU()
    (6): Linear(in_features=9, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [38]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2502601743
	Test error	Accuracy: 0.3972602785	Loss: 0.2502049201
Epoch 2 / 150
	Training	Accuracy: 0.4117647111	Loss: 0.2501519322
	Test error	Accuracy: 0.3972602785	Loss: 0.2501047516
Epoch 3 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2500397265
	Test error	Accuracy: 0.3972602785	Loss: 0.2500037856
Epoch 4 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2499230951
	Test error	Accuracy: 0.3972602785	Loss: 0.2498992192
Epoch 5 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2498037666
	Test error	Accuracy: 0.4109589159	Loss: 0.2497923474
Epoch 6 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2496735007
	Test error	Accuracy: 0.4109589159	Loss: 0.2496847454
Epoch 7 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2495396882
	Test error	Accuracy: 0.4109589159	Loss: 0.2495722288
Epoch 8 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2494028360
	Test error	Accuracy: 0.4109589159	Loss: 0.2494553022
Epoch 9 / 150
	Training	Accuracy: 0.4411

	Training	Accuracy: 0.8823529482	Loss: 0.2215581536
	Test error	Accuracy: 0.9315068722	Loss: 0.2233721280
Epoch 71 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2207271457
	Test error	Accuracy: 0.9315068722	Loss: 0.2225999527
Epoch 72 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2198849320
	Test error	Accuracy: 0.9315068722	Loss: 0.2218250824
Epoch 73 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2190261036
	Test error	Accuracy: 0.9315068722	Loss: 0.2210277227
Epoch 74 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2181758881
	Test error	Accuracy: 0.9315068722	Loss: 0.2202407973
Epoch 75 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2173036039
	Test error	Accuracy: 0.9315068722	Loss: 0.2194319438
Epoch 76 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2164473385
	Test error	Accuracy: 0.9315068722	Loss: 0.2186220061
Epoch 77 / 150
	Training	Accuracy: 0.8823529482	Loss: 0.2155732512
	Test error	Accuracy: 0.9315068722	Loss: 0.2177937272
Epoch 78 / 150
	Training	Accuracy: 0.8823529482

	Training	Accuracy: 0.9411764741	Loss: 0.1517109573
	Test error	Accuracy: 0.9589040875	Loss: 0.1580459241
Epoch 139 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1505869180
	Test error	Accuracy: 0.9589040875	Loss: 0.1569961317
Epoch 140 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1494455040
	Test error	Accuracy: 0.9589040875	Loss: 0.1559153079
Epoch 141 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1483201981
	Test error	Accuracy: 0.9589040875	Loss: 0.1548478696
Epoch 142 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1472127289
	Test error	Accuracy: 0.9589040875	Loss: 0.1538040489
Epoch 143 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1460873634
	Test error	Accuracy: 0.9589040875	Loss: 0.1527424242
Epoch 144 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1449512243
	Test error	Accuracy: 0.9589040875	Loss: 0.1516672529
Epoch 145 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1438183635
	Test error	Accuracy: 0.9589040875	Loss: 0.1505941294
Epoch 146 / 150
	Training	Accuracy: 0.94

### Prepare results for analysis

In [39]:
tailend = f"ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save ReLU Metrics to Tensorboard <a id="relu-metrics-for-tensorboard"></a>

In [40]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ReLU predictions to persistent storage

In [41]:
df["pred_relu"] = np.array(np.round(test_preds[-1])).astype(int)

name = f"{prefix}_relu_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_relu_scaled.pth")

compas_relu_mk2.pth has been saved.


In [42]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target,pred_relu
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1.0,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1.0,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0.0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1.0,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1.0,1


## Trial 3.2 - Tanh (COMPAS) <a id="tanh"></a>

#### Hyperparameters

In [43]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = tanh
hidden2 = tanh
hidden3 = tanh
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [44]:
# instantiate model
model = MLP(X.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
loss_function = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=33, out_features=22, bias=True)
    (1): Tanh()
    (2): Linear(in_features=22, out_features=14, bias=True)
    (3): Tanh()
    (4): Linear(in_features=14, out_features=9, bias=True)
    (5): Tanh()
    (6): Linear(in_features=9, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Training and test

In [45]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2516943216
	Test error	Accuracy: 0.3972602785	Loss: 0.2471698714
Epoch 2 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2505333126
	Test error	Accuracy: 0.3972602785	Loss: 0.2460985801
Epoch 3 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2493666708
	Test error	Accuracy: 0.3972602785	Loss: 0.2450114886
Epoch 4 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2481865585
	Test error	Accuracy: 0.3972602785	Loss: 0.2439193030
Epoch 5 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2469901145
	Test error	Accuracy: 0.3972602785	Loss: 0.2428214011
Epoch 6 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2457959205
	Test error	Accuracy: 0.3972602785	Loss: 0.2417160713
Epoch 7 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2445748299
	Test error	Accuracy: 0.3972602785	Loss: 0.2405944247
Epoch 8 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2433530092
	Test error	Accuracy: 0.3972602785	Loss: 0.2394660740
Epoch 9 / 150
	Training	Accuracy: 0.4411

	Training	Accuracy: 0.9411764741	Loss: 0.1493134201
	Test error	Accuracy: 0.9315068722	Loss: 0.1545055927
Epoch 71 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1477558613
	Test error	Accuracy: 0.9315068722	Loss: 0.1530675704
Epoch 72 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1461754590
	Test error	Accuracy: 0.9452054501	Loss: 0.1516236918
Epoch 73 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1446318328
	Test error	Accuracy: 0.9452054501	Loss: 0.1501869687
Epoch 74 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1430760026
	Test error	Accuracy: 0.9452054501	Loss: 0.1487505436
Epoch 75 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1415139884
	Test error	Accuracy: 0.9452054501	Loss: 0.1473044256
Epoch 76 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1399683207
	Test error	Accuracy: 0.9452054501	Loss: 0.1458783902
Epoch 77 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1384368241
	Test error	Accuracy: 0.9452054501	Loss: 0.1444598749
Epoch 78 / 150
	Training	Accuracy: 0.9411764741

	Training	Accuracy: 0.9705882072	Loss: 0.0663761869
	Test error	Accuracy: 0.9726027250	Loss: 0.0738699298
Epoch 139 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0656458959
	Test error	Accuracy: 0.9726027250	Loss: 0.0731073171
Epoch 140 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0649189129
	Test error	Accuracy: 0.9726027250	Loss: 0.0723493521
Epoch 141 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0641931072
	Test error	Accuracy: 0.9726027250	Loss: 0.0715801684
Epoch 142 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0634836629
	Test error	Accuracy: 0.9726027250	Loss: 0.0708324232
Epoch 143 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0627916157
	Test error	Accuracy: 0.9726027250	Loss: 0.0701053164
Epoch 144 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0621101297
	Test error	Accuracy: 0.9726027250	Loss: 0.0693844832
Epoch 145 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0614408478
	Test error	Accuracy: 0.9726027250	Loss: 0.0686699132
Epoch 146 / 150
	Training	Accuracy: 0.97

#### Prepare tanh results for analysis

In [46]:
tailend = f"Tanh ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Generate tensorboard summary

In [47]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Store Tanh results in dataframe 

In [48]:
df["pred_tanh"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_tanh_{suffix}.pth"
custom_save(name, model)
#torch.save(model, f"{prefix}_tanh_scaled.pth")

compas_tanh_mk2.pth has been saved.


In [49]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target,pred_relu,pred_tanh
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1.0,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1.0,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0.0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1.0,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1.0,1,1


## Trial 3.3 - ELU (COMPAS)<a id="elu"></a>

#### Hyperparameters

In [50]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = elu
hidden2 = elu
hidden3 = elu
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct model

In [51]:
# instantiate model
model = MLP(X.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=33, out_features=22, bias=True)
    (1): ELU(alpha=1.0)
    (2): Linear(in_features=22, out_features=14, bias=True)
    (3): ELU(alpha=1.0)
    (4): Linear(in_features=14, out_features=9, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=9, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [52]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                            test_s,
                                                            model,
                                                            loss_function,
                                                            metric,
                                                            optimizer,
                                                            epochs,
                                                            early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2526420057
	Test error	Accuracy: 0.3150684834	Loss: 0.2532411019
Epoch 2 / 150
	Training	Accuracy: 0.4705882370	Loss: 0.2519218028
	Test error	Accuracy: 0.3561643958	Loss: 0.2525221010
Epoch 3 / 150
	Training	Accuracy: 0.4705882370	Loss: 0.2511947453
	Test error	Accuracy: 0.3835616410	Loss: 0.2518041992
Epoch 4 / 150
	Training	Accuracy: 0.5000000000	Loss: 0.2504802644
	Test error	Accuracy: 0.3972602785	Loss: 0.2510956022
Epoch 5 / 150
	Training	Accuracy: 0.5000000000	Loss: 0.2497570366
	Test error	Accuracy: 0.4109589159	Loss: 0.2503848005
Epoch 6 / 150
	Training	Accuracy: 0.5000000000	Loss: 0.2490449250
	Test error	Accuracy: 0.4657534361	Loss: 0.2496732757
Epoch 7 / 150
	Training	Accuracy: 0.5294117928	Loss: 0.2483220994
	Test error	Accuracy: 0.4657534361	Loss: 0.2489615757
Epoch 8 / 150
	Training	Accuracy: 0.6176470518	Loss: 0.2475927025
	Test error	Accuracy: 0.5068492889	Loss: 0.2482390226
Epoch 9 / 150
	Training	Accuracy: 0.6176

	Training	Accuracy: 0.8235294223	Loss: 0.1708803773
	Test error	Accuracy: 0.8767123222	Loss: 0.1749165278
Epoch 71 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1693591326
	Test error	Accuracy: 0.8767123222	Loss: 0.1735014447
Epoch 72 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1678642780
	Test error	Accuracy: 0.8767123222	Loss: 0.1721147199
Epoch 73 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1663591862
	Test error	Accuracy: 0.8767123222	Loss: 0.1707311989
Epoch 74 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1648470759
	Test error	Accuracy: 0.8767123222	Loss: 0.1693159760
Epoch 75 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1633389741
	Test error	Accuracy: 0.8767123222	Loss: 0.1679333079
Epoch 76 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.1618376672
	Test error	Accuracy: 0.8767123222	Loss: 0.1665542026
Epoch 77 / 150
	Training	Accuracy: 0.8529411554	Loss: 0.1603398025
	Test error	Accuracy: 0.8767123222	Loss: 0.1651600777
Epoch 78 / 150
	Training	Accuracy: 0.8529411554

	Training	Accuracy: 0.9705882072	Loss: 0.0730753541
	Test error	Accuracy: 0.9726027250	Loss: 0.0835473552
Epoch 139 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0719746724
	Test error	Accuracy: 0.9726027250	Loss: 0.0824641259
Epoch 140 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0709027573
	Test error	Accuracy: 0.9726027250	Loss: 0.0813964403
Epoch 141 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0698432326
	Test error	Accuracy: 0.9726027250	Loss: 0.0803494510
Epoch 142 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0688063353
	Test error	Accuracy: 0.9726027250	Loss: 0.0793072996
Epoch 143 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0677824616
	Test error	Accuracy: 0.9726027250	Loss: 0.0782870232
Epoch 144 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0667916983
	Test error	Accuracy: 0.9726027250	Loss: 0.0772899646
Epoch 145 / 150
	Training	Accuracy: 0.9705882072	Loss: 0.0658110082
	Test error	Accuracy: 0.9726027250	Loss: 0.0762980829
Epoch 146 / 150
	Training	Accuracy: 0.97

#### Prepare ELU results for analysis

In [53]:
tailend = f"ELU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Save metrics to tensorboad

In [54]:
to_tensorboard(tailend, directory, training_s, train_preds, test_preds, y_train, y_test)

#### Add ELU test results to persistent storage

In [55]:
df["pred_elu"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_elu_{suffix}.pth"
custom_save(name, model, 1)
#torch.save(model, f"{prefix}_elu_scaled.pth")

compas_elu_mk2.pth has been saved.


In [56]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target,pred_relu,pred_tanh,pred_elu
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1.0,1,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1.0,1,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0.0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1.0,1,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1.0,1,1,1


## Trial 3.4 - Leaky ReLU (COMPAS) <a id="leaky"></a>

#### Hyperparameters

In [57]:
#alpha = 1e-5 # 0.00001
#epochs = 50
#batches = 256
hidden1 = leaky
hidden2 = leaky
hidden3 = leaky
out_layer = sigmoid
print(f"learning rate: {alpha}")
print(f"epochs: {epochs}")
print(f"batches: {batches}")

learning rate: 1e-05
epochs: 150
batches: 100


#### Construct Model

In [58]:
# instantiate model
model = MLP(X.shape[1], hidden1, hidden2, hidden3, out_layer).to(device)

# select optimizing function
optimizer = adam(model.parameters(), lr=alpha)

# select loss function
criterion = mse

# accuracy metric
metric = torchmetrics.functional.accuracy

# display model
print(model.parameters)

<bound method Module.parameters of MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=33, out_features=22, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=22, out_features=14, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=14, out_features=9, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=9, out_features=1, bias=True)
    (7): Sigmoid()
  )
)>


#### Train and test

In [59]:
training_results = []
training_losses = []
testing_results = []

training_results, training_losses, testing_results = evaluate(training_s,
                                                             test_s,
                                                             model,
                                                             loss_function,
                                                             metric,
                                                             optimizer,
                                                             epochs,
                                                             early_stop=False)

Epoch 1 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2531914413
	Test error	Accuracy: 0.3972602785	Loss: 0.2523242725
Epoch 2 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2530167401
	Test error	Accuracy: 0.3972602785	Loss: 0.2521587922
Epoch 3 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2528381944
	Test error	Accuracy: 0.3972602785	Loss: 0.2519911201
Epoch 4 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2526576817
	Test error	Accuracy: 0.3972602785	Loss: 0.2518205040
Epoch 5 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2524733245
	Test error	Accuracy: 0.3972602785	Loss: 0.2516428339
Epoch 6 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2522877157
	Test error	Accuracy: 0.3972602785	Loss: 0.2514602400
Epoch 7 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2520974874
	Test error	Accuracy: 0.3972602785	Loss: 0.2512701076
Epoch 8 / 150
	Training	Accuracy: 0.4411764741	Loss: 0.2518987656
	Test error	Accuracy: 0.3972602785	Loss: 0.2510720307
Epoch 9 / 150
	Training	Accuracy: 0.4411

	Training	Accuracy: 0.7647058964	Loss: 0.2129469216
	Test error	Accuracy: 0.8356164098	Loss: 0.2143624148
Epoch 71 / 150
	Training	Accuracy: 0.7647058964	Loss: 0.2119623870
	Test error	Accuracy: 0.8356164098	Loss: 0.2134389366
Epoch 72 / 150
	Training	Accuracy: 0.7941176295	Loss: 0.2109721899
	Test error	Accuracy: 0.8356164098	Loss: 0.2125118212
Epoch 73 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2099698633
	Test error	Accuracy: 0.8356164098	Loss: 0.2115772806
Epoch 74 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2089601457
	Test error	Accuracy: 0.8356164098	Loss: 0.2106316239
Epoch 75 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2079326361
	Test error	Accuracy: 0.8356164098	Loss: 0.2096784945
Epoch 76 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2069185078
	Test error	Accuracy: 0.8356164098	Loss: 0.2087207919
Epoch 77 / 150
	Training	Accuracy: 0.8235294223	Loss: 0.2058893144
	Test error	Accuracy: 0.8356164098	Loss: 0.2077545623
Epoch 78 / 150
	Training	Accuracy: 0.8529411554

	Training	Accuracy: 0.9411764741	Loss: 0.1378590912
	Test error	Accuracy: 0.9589040875	Loss: 0.1432162938
Epoch 139 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1367232352
	Test error	Accuracy: 0.9589040875	Loss: 0.1421253546
Epoch 140 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1356220543
	Test error	Accuracy: 0.9589040875	Loss: 0.1410347897
Epoch 141 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1345204413
	Test error	Accuracy: 0.9589040875	Loss: 0.1399458388
Epoch 142 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1334127486
	Test error	Accuracy: 0.9589040875	Loss: 0.1388452660
Epoch 143 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1323233694
	Test error	Accuracy: 0.9726027250	Loss: 0.1377617337
Epoch 144 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1312430799
	Test error	Accuracy: 0.9726027250	Loss: 0.1366901834
Epoch 145 / 150
	Training	Accuracy: 0.9411764741	Loss: 0.1301447302
	Test error	Accuracy: 0.9726027250	Loss: 0.1355740957
Epoch 146 / 150
	Training	Accuracy: 0.94

#### Prepare Leaky ReLU data for analysis

In [60]:
tailend = f"Leaky ReLU ({prefix})"

train_preds, test_preds = prep(tailend, training_results, testing_results)

#### Add Leaky ReLU data to persistent storage

In [61]:
df["pred_leaky"] = np.array(np.round(test_preds[-1])).astype(int)
name = f"{prefix}_leaky_{suffix}.pth"
custom_save(name, model,1)
# torch.save(model, "loans_leaky_scaled.pth")

compas_leaky_mk2.pth has been saved.


In [62]:
df.head()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,decile_score.1,v_decile_score,priors_count.1,start,end,sex,age_cat_25to45,age_cat_over45,age_cat_under25,race_black,race_asian,race_white,race_hispanic,race_native,race_Other,c_charge_degree,score_text_High,score_text_Low,score_text_Medium,v_score_text_High,v_score_text_Low,v_score_text_Medium,event,target,pred_relu,pred_tanh,pred_elu,pred_leaky
0,25,0,5,0,1,4,-1,1,1,0,5,4,4,32,304,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,1,1.0,1,1,1,1
1,35,0,4,0,0,3,-1,0,1,0,4,4,3,9,462,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1.0,1,1,1,1
2,32,0,4,0,0,1,0,0,0,0,4,3,1,0,1171,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0.0,0,0,0,0
3,20,0,10,0,0,0,-1,1,1,1,10,10,0,3,20,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,1.0,1,1,1,1
4,50,0,8,0,2,24,0,1,1,1,8,8,24,32,87,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1.0,1,1,1,1


# Results

## Save data

In [63]:
name = f"{prefix}_results_{suffix}.parquet"
custom_save(name, df, 0)

compas_results_mk2.parquet has been saved.
